# Install Dependencies:

In [ ]:
!python --version

In [ ]:
!pip install ultralytics
!wandb disabled

In [ ]:
import torch
torch.__version__

In [ ]:
!pip install pylabel
from pylabel import importer

# Preparing Data:

In [ ]:
!cp -r "/kaggle/input/car-license-plate-detection-iran" "./images"
!cp "/kaggle/input/car-license-detection/plate_yolov8_2.yaml" ./
!cp "/kaggle/input/car-license-detection/yolov8n.pt/yolov8n.pt" ./
!mkdir data
!mkdir labels

In [ ]:
dataset = importer.ImportCoco(path="./images/test/_annotations.coco.json",
                              path_to_images="./images/test")
dataset.export.ExportToYoloV5("labels/test")

In [ ]:
dataset = importer.ImportCoco(path="./images/train/_annotations.coco.json",
                              path_to_images="./images/train")
dataset.export.ExportToYoloV5("labels/train")

In [ ]:
dataset = importer.ImportCoco(path="./images/valid/_annotations.coco.json",
                              path_to_images="./images/valid")
dataset.export.ExportToYoloV5("labels/val")

In [ ]:
!mv images data
!mv labels data
!mv data/images/valid data/images/val
!rm data/images/train/_annotations.coco.json
!rm data/images/test/_annotations.coco.json
!rm data/images/val/_annotations.coco.json

In [ ]:
!ls data/images/train | wc -l

In [ ]:
!ls data/images/test | wc -l

In [ ]:
!ls data/images/val | wc -l

In [ ]:
with open("plate_yolov8_2.yaml", 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write("path: /kaggle/working" + '\n' + content)

# Train:

In [ ]:
import time
from ultralytics import YOLO
model.train(data="./plate_yolov8_2.yaml", epochs=10, imgsz=640, batch=64)

In [ ]:
tempos = []
for i in range(30):
    model = YOLO("./yolov8n.pt")
    inicio = time.time()
    model.train(data="./plate_yolov8_2.yaml", epochs=10, imgsz=320, batch=32) # imgsz = 320 e 640;  batch = 32 e 64
    fim = time.time()
    tempo_total = fim - inicio
    tempos.append(tempo_total)
    print(i+1, f"Tempo levado: {tempo_total:.6f} segundos")
    !rm -r runs
    !rm data/images/train/*.npy
    !rm data/images/val/*.npy
    !rm data/labels/*.cache
    del model
    gc.collect()

In [ ]:
tempos

# Test:

In [ ]:
!ls "./data/images/test/"

In [ ]:
!yolo val model="./runs/detect/train/weights/best.pt" data="./plate_yolov8_2.yaml" conf=0.1 iou=0.65 plots=True split=test

# Predict:

In [ ]:
model = YOLO("./runs/detect/train/weights/best.pt")

In [ ]:
res = model("./data/images/test/car-101-_jpg.rf.c1af2d5503b6bba559d1e8f3fcdcf3f7.jpg")

In [ ]:
res